<a href="https://colab.research.google.com/github/HugoTHO/sisbi-utilities-notebooks/blob/main/sigaa_register_daily_movement_of_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTOMAÇÃO DE REGISTRO DE MOVIMENTAÇÃO DIÁRIA DE USUÁRIOS
***

## Importação de bibliotecas Python

In [ ]:
import bs4
import json
import re
import requests
import urllib.parse as urlparser

from getpass import getpass
from ipywidgets import widgets
from html import unescape

## Login

In [ ]:
#@markdown Execute esta célula e forneça sua credencial do Sigaa e senha.
#@markdown Em seguida, escolha seu vínculo. <hr>
#@markdown O login no sistema e escolha do vínculo são necessários para as 
#@markdown próximas operações.

base_url = "https://sigaa.ufrn.br"
base_path = "/sigaa/"
relship_sel_path = "/sigaa/vinculos.jsf"

# INTERFACE UTILITIES
dropbox_placeholder = "SELECIONE O VINCULO"
default_box_lt = widgets.Layout(padding='0 0 28px 0', border='1px solid white')
success_box_lt = widgets.Layout(padding='0 0 28px 0', border='1px solid green')

session = requests.Session()
login_page = session.get(urlparser.urljoin(base_url, base_path))

# Checks if a valid user is already logged
try:
    username
except NameError:
    username = input("Usuário: ")
    password = getpass("Senha: ")

# Mounts the POST necessary login data
login_form = (
    bs4.BeautifulSoup(login_page.text)
    .find('form', {'id':'login-form'})
)
login_path = login_form['action']
login_data = {
    'username':username,
    'password':password
}
for hidden_input in login_form.find_all('input',{'type':'hidden'}):
    login_data[hidden_input['name']] = hidden_input['value']

response = session.post(
    urlparser.urljoin(login_page.url,login_path),
    data=login_data
)

if len(response.history) and response.history[0].status_code > 300:
    print('Login realizado com sucesso, escolha seu vínculo:')

    relationship_page = session.get(
        urlparser.urljoin(base_url, relship_sel_path)
    )
    relship_links = (
        bs4.BeautifulSoup(relationship_page.text)
        .find_all('a', class_='withoutFormat')
    )

    # Concatenates the relationship type and description on link list,
    # converting the escaped charactes.
    choices = [
        (reltype.string + 
        unescape(relship.string.split(':')[1]
            .encode('ascii', 'xmlcharrefreplace')
            .decode('utf-8'))
        )
        for reltype, relship in zip(
            relship_links[::3],
            relship_links[2::3]
        )
    ]

    relship_dd = widgets.Dropdown(
        options=[dropbox_placeholder] + choices,
        value=dropbox_placeholder,
        layout=default_box_lt
    )
    display(relship_dd)

    # Registers a handler function to access the choiced relationship
    def dd_change(change):
        if (change['new'] != 0):
            relship_path = relship_links[(change['new']-1)*3]['href']
            session.get(urlparser.urljoin(base_url, relship_path))
            relship_dd.layout = success_box_lt
        else:
            relship_dd.layout = default_box_lt

    relship_dd.observe(dd_change, names='index')

else:
    print('Problema ao realizar o login')
    # deletes the username variable to reinsert the credentials
    del username

## Cadastrar movimentação diária

In [25]:
#@title Escolher biblioteca, mês e ano
#@markdown <hr> 
#@markdown Execute essa célula paara visualizar e escolher entre as opções
#@markdown disponíveis

library_path = "/sigaa/biblioteca/index.jsf"
page_link_str = "Cadastrar Movimentação Diária de Usuários"

month_list = ["Janeiro", "Fevereiro", "Março", "Abril",
              "Maio", "Junho", "Julho", "Agosto",
              "Setembro", "Outubro", "Novembro", "Dezembro"]


def get_jsfcl_path(command_link):
    '''Given a JSF command_link bs4.tag, returns path to access 
    the next page.
    '''
    return link_tag.find_parent('form')['action']


def get_jsfcl_post_data(command_link):
    '''Given a JSF command_link bs4.tag, returns the post data to 
    access the next page.
    '''
    post_data = { tag['name']: tag['value'] 
        if tag.has_attr('value') 
        else ""
        for tag in link_tag.find_parent('form').find_all('input')
    }

    # Gets the jsfcljs javascript function's json param on JSF command link's
    # onclick attr that references next page.
    #
    # more info:
    # https://jakarta.ee/specifications/faces/2.3/jsdoc/jsf-uncompressed.js.html
    re_pattern_str = r"""jsfcljs\(                            # function call
                         (?P<form>.+?),                       # form param
                         (?P<json>{(?:(?:.+?):(?:.+?),?)+?}), # json param
                         (?:(?P<target>.+?)\))                # target param"""
    post_data.update(
        json.loads(
            re.search(
                pattern=re_pattern_str,
                string=link_tag['onclick'],
                flags=re.VERBOSE,
            ).group('json').replace("'",'"')
        )
    )
    return post_data


def display_options(choices_dict):
    '''TODO: document this function
    '''
    choice_wgets = {}
    # Basic Widgets
    choice_wgets['month'] = widgets.Dropdown(
        options = month_list,
        value = month_list[int(choices_dict['month']['value'])-1]
    )
    choice_wgets['year'] = widgets.IntText(choices_dict['year']['value'])
    choice_wgets['library'] = widgets.Dropdown(
        options = [option['string']
            for option in choices_dict['library']['options']
        ],
        value = choices_dict['library']['options'][0]['string']
    )

    # Containers
    label_vb = widgets.VBox([
        widgets.Label("Mês da Frequência:"),
        widgets.Label("Ano da Frequência:"),
        widgets.Label("Biblioteca:")
    ])
    widgets_vb = widgets.VBox([
        choice_wgets['month'],
        choice_wgets['year'],
        choice_wgets['library']
    ])

    display(widgets.HBox([label_vb, widgets_vb]))
    return choice_wgets


library_page = session.get(urlparser.urljoin(base_url, library_path))
link_tag = (
    bs4.BeautifulSoup(library_page.text)
    .find('a', string=page_link_str)
)

lib_month_year_page = session.post(
    urlparser.urljoin(base_url, get_jsfcl_path(link_tag)),
    data=get_jsfcl_post_data(link_tag)
)

options_form = (bs4.BeautifulSoup(lib_month_year_page.text)
    .find('form', id='formulario'))

form_data = {}

library_choices = None
for form_field in (options_form.find_all('th')):
    if 'Mês' in form_field.string:
        form_data['month'] = {
            'name': form_field.findNext('input')['name'],
            'value': form_field.findNext('input')['value']
        }
    elif 'Ano' in form_field.string:
        form_data['year'] = {
            'name': form_field.findNext('input')['name'],
            'value': form_field.findNext('input')['value']
        }
    else:
        form_data['library'] = {
            'name': form_field.find_next('select')['name'],
            'options': [{
                'string': option.string,
                'value': option['value']
                } for option in (form_field
                    .find_next('select')
                    .find_all('option')
                )
            ]
        }

choice_wgets = display_options(form_data)

In [32]:
btn_value = 'Consultar'


post_data = {input['name']: input['value'] 
    for input in options_form.find_all('input', type='hidden')
}
post_data[options_form.find('input', value=btn_value)['name']] = btn_value

# TODO Comment this block
post_data[form_data['library']['name']] = (form_data
    .get('library')
    .get('options')[
        choice_wgets['library'].index
    ].get('value')
)
post_data[form_data['month']['name']] = choice_wgets['month'].index + 1
post_data[form_data['year']['name']] = choice_wgets['year'].value
    

movement_register_page = session.post(
    urlparser.urljoin(base_url, options_form['action']),
    data=post_data
)

In [ ]:
import IPython
IPython.display.HTML(movement_register_page.text)